In [ ]:

import os
import numpy as np
import glob
import warnings
import nltk
import string
import collections
import pandas as pd
from shutil import copyfile


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Caricamento dati di train e test che verranno poi uniti

In [ ]:
copyfile('/content/gdrive/MyDrive/Text Mining/TM_Project/test_neg.csv', 'test_neg.csv')
copyfile('/content/gdrive/MyDrive/Text Mining/TM_Project/train_neg.csv', 'train_neg.csv')

'train_neg.csv'

In [ ]:
test_neg=pd.read_csv('test_neg.csv')
train_neg=pd.read_csv('train_neg.csv')

In [ ]:
neg= train_neg.append(test_neg).reset_index()
neg.rename(columns = {'0' : 'text'}, inplace = True)

In [ ]:
neg.drop(['index','Unnamed: 0'],axis=1, inplace=True)
neg

,text
0,"['harold', 'pinter', 'rewrite', 'anthony', 'sc..."
1,"['somebody', 'could', 'probably', 'make', 'gre..."
2,"['oh', 'dear', 'rarely', 'ha', 'horror', 'film..."
3,"['must', 'preface', 'saying', 'huge', 'romanti..."
4,"['movie', 'wife', 'ever', 'walked', 'totally',..."
...,...
24995,"['evil', 'start', 'strait', 'san', 'sebastian'..."
24996,"['disposable', 'movie', 'history', 'cinema', '..."
24997,"['woman', '2008', 'diane', 'english', 'sadly',..."
24998,"['mild', 'spoiler', 'would', 'satire', 'chapli..."


Caricamento di alcune funzioni utili per il preprocessing

In [ ]:
!wget -q "https://www.dropbox.com/s/yfjx9nhrky1ifws/preprocessing_functions.py" -O preprocessing_functions.py

In [ ]:
 from preprocessing_functions import remove_punctuation, remove_numbers, remove_stopwords,remove_extra_whitespace

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
neg.text = neg.text.apply(remove_punctuation)
neg.text = neg.text.apply(remove_extra_whitespace)
neg.text = neg.text.apply(remove_numbers)

In [ ]:
# Don't print warnings
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn

In [ ]:
neg

,text
0,harold pinter rewrite anthony schaeffer classi...
1,somebody could probably make great documentary...
2,oh dear rarely ha horror film bored made laugh...
3,must preface saying huge romantic hence really...
4,movie wife ever walked totally sucked saw omah...
...,...
24995,evil start strait san sebastian th century tr...
24996,disposable movie history cinema one strong con...
24997,woman diane english sadly waste talent annett...
24998,mild spoiler would satire chaplin set sight ev...


In [ ]:
!pip install -q pyLDAvis==2.1.2

In [ ]:
from __future__ import print_function

In [ ]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

Topic Modelling with gensim: creating the dictionary and then using the LDA model

In [ ]:
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

In [ ]:
texts = list(neg.text.str.split())

In [ ]:
dictionary = Dictionary(texts)
dictionary.filter_extremes(no_below=7, no_above=0.3)

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]

* Fit LDA model


In [ ]:
lda_gensim = LdaModel(corpus, num_topics=8)

In [ ]:
# Print the Keyword in the 8 topics
lda_gensim.print_topics()


In [ ]:
#Getting the words from the dictionary
dictionary[84] 

In [ ]:
doc_lda = lda_gensim[corpus]

In [ ]:
pyLDAvis.gensim.prepare(lda_gensim, corpus, dictionary, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
6     -108.958252 -208.941483       1        1  19.897556
0      186.487061  -98.103249       2        1  18.211436
5       19.245832  160.858780       3        1  15.830583
4      170.658829   75.445076       4        1  12.524418
2       -1.386411  -35.043739       5        1  10.655619
1     -145.800095  106.944611       6        1  10.549813
3       64.834541 -221.825272       7        1   7.201925
7     -196.906036  -58.990913       8        1   5.128649, topic_info=       Term          Freq         Total Category  logprob  loglift
173    show   8263.000000   8263.000000  Default  30.0000  30.0000
192  horror   4230.000000   4230.000000  Default  29.0000  29.0000
767    book   3341.000000   3341.000000  Default  28.0000  28.0000
1    acting   7984.000000   7984.000000  Default  27.0000  27.0000
350   scene  11468.000000  11468.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
55     know    283.333834   7656.013753   Topic8  -6.1238  -0.3263
370   actor    277.476125   7407.209062   Topic8  -6.1447  -0.3141
360   thing    275.554128   9375.133785   Topic8  -6.1516  -0.5567
73    never    265.273214   6855.913741   Topic8  -6.1896  -0.2818
143   could    268.179116   9132.803086   Topic8  -6.1787  -0.5576

[788 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
1          1  0.549031  acting
1          2  0.226801  acting
1          3  0.051722  acting
1          4  0.026800  acting
1          5  0.029556  acting
...      ...       ...     ...
15643      6  0.938172       –
15643      8  0.007951       –
15645      4  0.008996       ’
15645      6  0.980546       ’
15645      8  0.004498       ’

[2553 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 1, 6, 5, 3, 2, 4, 8])

Evaluation metrics

In [ ]:
print('Perplexity: ',
lda_gensim.log_perplexity(corpus))

Perplexity:  -8.321950986947003


In [ ]:
from gensim.models import CoherenceModel

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_gensim, texts=texts, dictionary=dictionary, coherence='c_v')

In [ ]:
coherence_lda = coherence_model_lda.get_coherence()    
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.2830025522507358


In [ ]:
lda_5_topic = LdaModel(corpus, num_topics=5)
coherence_model_lda_5_topic = CoherenceModel(model=lda_5_topic, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda_5_topic = coherence_model_lda_5_topic.get_coherence()    
print('Coherence Score: ', coherence_lda_5_topic)

Coherence Score:  0.281894415081654


In [ ]:
lda_10_topic = LdaModel(corpus, num_topics=10)
coherence_model_lda_10_topic = CoherenceModel(model=lda_10_topic, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda_10_topic = coherence_model_lda_10_topic.get_coherence()    
print('Coherence Score: ', coherence_lda_10_topic)

Coherence Score:  0.28257964537240055


In [ ]:
lda_15_topic = LdaModel(corpus, num_topics=15)
coherence_model_lda_15_topic = CoherenceModel(model=lda_15_topic, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda_15_topic = coherence_model_lda_15_topic.get_coherence()    
print('Coherence Score: ', coherence_lda_15_topic)

Coherence Score:  0.2813944133055876


In [ ]:
lda_19_topic = LdaModel(corpus, num_topics=19)
coherence_model_lda_19_topic = CoherenceModel(model=lda_19_topic, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda_19_topic = coherence_model_lda_19_topic.get_coherence()    
print('Coherence Score: ', coherence_lda_19_topic)

Coherence Score:  0.28419835833193013


In [ ]:
#pyLDAvis.gensim.prepare(lda_8_topic, corpus, dictionary, mds='tsne')

In [ ]:
#pyLDAvis.gensim.prepare(lda_5_topic, corpus, dictionary, mds='tsne')